# EDA for Enefit

https://www.kaggle.com/competitions/predict-energy-behavior-of-prosumers

## Feature Engineering
https://www.kaggle.com/code/rafiko1/enefit-xgboost-starter

## To understand the data

https://www.kaggle.com/code/vitalykudelya/explain-dataset-and-baseline

## How to merge?
https://www.kaggle.com/competitions/predict-energy-behavior-of-prosumers/discussion/455100

# Import Libraries

In [102]:
from pathlib import Path
import os
import warnings
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import gc  # Garbage collection for memory management
import time  # Time-related functions
import warnings  # Handling warnings
from itertools import combinations  # For creating combinations of elements
from warnings import simplefilter  # Simplifying warning handling
import joblib  # For saving and loading models
import numpy as np  # Numerical operations
import pandas as pd  # Data manipulation and analysis
from sklearn.metrics import mean_absolute_error  # Metric for evaluation
from sklearn.model_selection import KFold, TimeSeriesSplit  # Cross-validation techniques
from concurrent.futures import ThreadPoolExecutor
from numba import njit, prange  # Compiling Python code for performance
from dataclasses import dataclass
import polars as pl
from gc import collect;
from psutil import Process;
from os import system, getpid, walk;
import torch

# Disable warnings to keep the code clean
warnings.filterwarnings("ignore")
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)


# Dataclass for holding data

In [103]:
# DataPath
@dataclass(frozen=True)
class DataPath:
    base_path: str
    train: str = None
    client: str = None
    gas_prices: str = None
    electricity_prices: str = None
    forecast_weather: str = None
    historical_weather: str = None
    weather_station_to_county_mapping: str = None
    county_lon_lats: str = None
    county_id_to_name_map: str = None

    def __post_init__(self):
        predict_energy_behavior_of_prosumers_path = os.path.join(self.base_path, "predict-energy-behavior-of-prosumers")
        fabiendaniels_mapping_locations_and_county_codes_path = os.path.join(self.base_path, "fabiendaniels-mapping-locations-and-county-codes")

        object.__setattr__(self, 'train', os.path.join(predict_energy_behavior_of_prosumers_path, "train.csv"))
        object.__setattr__(self, 'client', os.path.join(predict_energy_behavior_of_prosumers_path, "client.csv"))
        object.__setattr__(self, 'gas_prices', os.path.join(predict_energy_behavior_of_prosumers_path, "gas_prices.csv"))
        object.__setattr__(self, 'electricity_prices', os.path.join(predict_energy_behavior_of_prosumers_path, "electricity_prices.csv"))
        object.__setattr__(self, 'forecast_weather', os.path.join(predict_energy_behavior_of_prosumers_path, "forecast_weather.csv"))
        object.__setattr__(self, 'historical_weather', os.path.join(predict_energy_behavior_of_prosumers_path, "historical_weather.csv"))
        object.__setattr__(self, 'weather_station_to_county_mapping', os.path.join(predict_energy_behavior_of_prosumers_path, "weather_station_to_county_mapping.csv"))
        object.__setattr__(self, 'county_lon_lats', os.path.join(fabiendaniels_mapping_locations_and_county_codes_path, "county_lon_lats.csv"))
        object.__setattr__(self, 'county_id_to_name_map', os.path.join(predict_energy_behavior_of_prosumers_path, "county_id_to_name_map.json"))
        

In [104]:
# Global params

In [132]:
seed = 42

# GPU or CPU use for model
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

# For kaggle environment
if os.environ.get("KAGGLE_DATA_PROXY_TOKEN") != None:
    IS_LOCAL = False
    BASE_INPUT_PATH = "/kaggle/input"
   
# For local environment
else:
    IS_LOCAL = True
    BASE_INPUT_PATH = "../kaggle/input"

data_path = DataPath(BASE_INPUT_PATH)
print("data_path", data_path)

data_path DataPath(base_path='../kaggle/input', train='../kaggle/input/predict-energy-behavior-of-prosumers/train.csv', client='../kaggle/input/predict-energy-behavior-of-prosumers/client.csv', gas_prices='../kaggle/input/predict-energy-behavior-of-prosumers/gas_prices.csv', electricity_prices='../kaggle/input/predict-energy-behavior-of-prosumers/electricity_prices.csv', forecast_weather='../kaggle/input/predict-energy-behavior-of-prosumers/forecast_weather.csv', historical_weather='../kaggle/input/predict-energy-behavior-of-prosumers/historical_weather.csv', weather_station_to_county_mapping='../kaggle/input/predict-energy-behavior-of-prosumers/weather_station_to_county_mapping.csv', county_lon_lats='../kaggle/input/fabiendaniels-mapping-locations-and-county-codes/county_lon_lats.csv', county_id_to_name_map='../kaggle/input/predict-energy-behavior-of-prosumers/county_id_to_name_map.json')


# Helper functions

In [144]:
from colorama import Fore, Style, init;

def PrintMemUsage():
    """
    This function defines the memory usage across the kernel. 
    Source-
    https://stackoverflow.com/questions/61366458/how-to-find-memory-usage-of-kaggle-notebook
    """;
    
    pid = getpid();
    py = Process(pid);
    memory_use = py.memory_info()[0] / 2. ** 30;
    print(f"RAM memory GB usage = {memory_use :.4}");

def get_pl_df_size(df: pl.DataFrame, unit: str = "gb"):
    # unit = {‘b’, ‘kb’, ‘mb’, ‘gb’, ‘tb’}
    mem_uage = df.estimated_size(unit)
    return mem_uage

# Helper functions
def display_df(df, name, rows = 5):
    '''Display df shape and rows'''
    PrintColor(text = f'{name} data has {df.shape[0]} rows and {df.shape[1]} columns. \n ===>  row:')
    display(df.head(rows))

# Color printing    
def PrintColor(text:str, color = Fore.BLUE, style = Style.BRIGHT):
    '''Prints color outputs using colorama of a text string'''
    print(style + color + text + Style.RESET_ALL);

# 🧹 Function to reduce memory usage of a Pandas DataFrame
def reduce_mem_usage(df, name: str, show_optimization: bool = False):
    """
    Iterate through all numeric columns of a dataframe and modify the data type
    to reduce memory usage.
    """
    
    # 📏 Calculate the initial memory usage of the DataFrame
    start_mem = df.memory_usage().sum() / 1024**2

    # 🔄 Iterate through each column in the DataFrame
    for col in df.columns:
        col_type = df[col].dtype

        # Check if the column's data type is not 'object' (i.e., numeric)
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            
            # Check if the column's data type is an integer
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                # Check if the column's data type is a float
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float32)

    if show_optimization:
        print(f"Memory usage of {name} is {start_mem:.2f} MB")
        end_mem = df.memory_usage().sum() / 1024**2
        print(f"Memory usage after optimization is: {end_mem:.2f} MB")
        decrease = 100 * (start_mem - end_mem) / start_mem
        print(f"Decreased by {decrease:.2f}%")

    # 🔄 Return the DataFrame with optimized memory usage

    start_mem = df.memory_usage().sum() / 1024**2
    return df


# Load Data

In [205]:
%%time
import json

train = pd.read_csv(data_path.train, parse_dates=True)
client = pd.read_csv(data_path.client, parse_dates=True)
gas = pd.read_csv(data_path.gas_prices, parse_dates=True)
electricity = pd.read_csv(data_path.electricity_prices, parse_dates=True)
forecast_weather = pd.read_csv(data_path.forecast_weather, parse_dates=True)
historical_weather = pd.read_csv(data_path.historical_weather, parse_dates=True)
weather_station_to_county_mapping = pd.read_csv(data_path.weather_station_to_county_mapping, parse_dates=True)
location = pd.read_csv(data_path.county_lon_lats, parse_dates=True).drop(columns = ["Unnamed: 0"])

COL_TARGET = "target"
col_train = train.columns
col_client = client.columns
col_gas = gas.columns
col_electricity = electricity.columns
col_forecast_weather = forecast_weather.columns
col_historical_weather = historical_weather.columns
col_weather_station_to_county_mapping = weather_station_to_county_mapping.columns
col_location = location.columns

# See county codes
with open(data_path.county_id_to_name_map) as f:
    county_codes = json.load(f)
pd.DataFrame(county_codes, index=[0])

train = reduce_mem_usage(train, "train")
client = reduce_mem_usage(client, "client")
gas = reduce_mem_usage(gas, "gas")
electricity = reduce_mem_usage(electricity, "electricity")
forecast_weather = reduce_mem_usage(forecast_weather, "forecast_weather")
historical_weather = reduce_mem_usage(historical_weather, "historical_weather")
weather_station_to_county_mapping = reduce_mem_usage(weather_station_to_county_mapping, "weather_station_to_county_mapping")
location = reduce_mem_usage(location, "location")

PrintMemUsage()

RAM memory GB usage = 1.243
CPU times: user 7.66 s, sys: 1.09 s, total: 8.76 s
Wall time: 9.22 s


In [115]:
"""
county - An ID code for the county.
is_business - Boolean for whether or not the prosumer is a business.
product_type - ID code with the following mapping of codes to contract types: {0: "Combined", 1: "Fixed", 2: "General service", 3: "Spot"}.
target - The consumption or production amount for the relevant segment for the hour. The segments are defined by the county, is_business, and product_type.
is_consumption - Boolean for whether or not this row's target is consumption or production.
datetime - The Estonian time in EET (UTC+2) / EEST (UTC+3). It describes the start of the 1-hour period on which target is given.
data_block_id - All rows sharing the same data_block_id will be available at the same forecast time. This is a function of what information is available when forecasts are actually made, at 11 AM each morning. For example, if the forecast weather data_block_id for predictins made on October 31st is 100 then the historic weather data_block_id for October 31st will be 101 as the historic weather data is only actually available the next day.
row_id - A unique identifier for the row.
prediction_unit_id - A unique identifier for the county, is_business, and product_type combination. New prediction units can appear or disappear in the test set.
"""

grouped_train

,county,is_business,product_type,count
0,0,0,1,30624
47,11,0,1,30624
33,7,1,1,30624
1,0,0,2,30624
35,8,0,1,30624
...,...,...,...,...
52,11,1,2,24000
50,11,1,0,23280
30,7,0,2,19968
45,10,1,2,12144


In [109]:
"""
product_type
county - An ID code for the county. See county_id_to_name_map.json for the mapping of ID codes to county names.
eic_count - The aggregated number of consumption points (EICs - European Identifier Code).
installed_capacity - Installed photovoltaic solar panel capacity in kilowatts.
is_business - Boolean for whether or not the prosumer is a business.
date
data_block_id
"""
client

,product_type,county,eic_count,installed_capacity,is_business,date,data_block_id
0,1,0,108,952.89,0,2021-09-01,2
1,2,0,17,166.40,0,2021-09-01,2
2,3,0,688,7207.88,0,2021-09-01,2
3,0,0,5,400.00,1,2021-09-01,2
4,1,0,43,1411.00,1,2021-09-01,2
...,...,...,...,...,...,...,...
41914,1,15,51,415.60,0,2023-05-29,637
41915,3,15,161,2035.75,0,2023-05-29,637
41916,0,15,15,620.00,1,2023-05-29,637
41917,1,15,20,624.50,1,2023-05-29,637


In [116]:
"""
[latitude/longitude] - The coordinates of the weather forecast.
origin_datetime - The timestamp of when the forecast was generated.
hours_ahead - The number of hours between the forecast generation and the forecast weather. Each forecast covers 48 hours in total.
temperature - The air temperature at 2 meters above ground in degrees Celsius. Estimated for the end of the 1-hour period.
dewpoint - The dew point temperature at 2 meters above ground in degrees Celsius. Estimated for the end of the 1-hour period.
cloudcover_[low/mid/high/total] - The percentage of the sky covered by clouds in the following altitude bands: 0-2 km, 2-6, 6+, and total. Estimated for the end of the 1-hour period.
10_metre_[u/v]_wind_component - The [eastward/northward] component of wind speed measured 10 meters above surface in meters per second. Estimated for the end of the 1-hour period.
data_block_id
forecast_datetime - The timestamp of the predicted weather. Generated from origin_datetime plus hours_ahead. This represents the start of the 1-hour period for which weather data are forecasted.
direct_solar_radiation - The direct solar radiation reaching the surface on a plane perpendicular to the direction of the Sun accumulated during the hour, in watt-hours per square meter.
surface_solar_radiation_downwards - The solar radiation, both direct and diffuse, that reaches a horizontal plane at the surface of the Earth, accumulated during the hour, in watt-hours per square meter.
snowfall - Snowfall over hour in units of meters of water equivalent.
total_precipitation - The accumulated liquid, comprising rain and snow that falls on Earth's surface over the described hour, in units of meters.

"""
forecast_weather

,latitude,longitude,origin_datetime,hours_ahead,temperature,dewpoint,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,data_block_id,forecast_datetime,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation
0,57.6,21.7,2021-09-01 02:00:00,1,15.655786,11.553613,0.904816,0.019714,0.000000,0.905899,-0.411328,-9.106137,1,2021-09-01 03:00:00,0.0,0.0,0.0,0.0
1,57.6,22.2,2021-09-01 02:00:00,1,13.003931,10.689844,0.886322,0.004456,0.000000,0.886658,0.206347,-5.355405,1,2021-09-01 03:00:00,0.0,0.0,0.0,0.0
2,57.6,22.7,2021-09-01 02:00:00,1,14.206567,11.671777,0.729034,0.005615,0.000000,0.730499,1.451587,-7.417905,1,2021-09-01 03:00:00,0.0,0.0,0.0,0.0
3,57.6,23.2,2021-09-01 02:00:00,1,14.844507,12.264917,0.336304,0.074341,0.000626,0.385468,1.090869,-9.163999,1,2021-09-01 03:00:00,0.0,0.0,0.0,0.0
4,57.6,23.7,2021-09-01 02:00:00,1,15.293848,12.458887,0.102875,0.088074,0.000015,0.176590,1.268481,-8.975766,1,2021-09-01 03:00:00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3424507,59.7,26.2,2023-05-30 02:00:00,48,10.150049,5.662744,0.061066,0.257339,0.004059,0.308594,6.890626,-3.258144,637,2023-06-01 02:00:00,0.0,0.0,0.0,0.0
3424508,59.7,26.7,2023-05-30 02:00:00,48,10.259790,5.922754,0.107880,0.895203,0.198700,0.906036,6.890626,-3.768032,637,2023-06-01 02:00:00,0.0,0.0,0.0,0.0
3424509,59.7,27.2,2023-05-30 02:00:00,48,10.467798,6.072534,0.000000,0.387466,0.448853,0.530518,6.214112,-4.700405,637,2023-06-01 02:00:00,0.0,0.0,0.0,0.0
3424510,59.7,27.7,2023-05-30 02:00:00,48,10.694116,7.009058,0.000000,0.505936,0.997314,0.997498,5.810060,-4.338345,637,2023-06-01 02:00:00,0.0,0.0,0.0,0.0


In [117]:
"""
datetime
temperature
dewpoint
rain - Different from the forecast conventions. The rain from large scale weather systems of the preceding hour in millimeters.
snowfall - Different from the forecast conventions. Snowfall over the preceding hour in centimeters.
surface_pressure - The air pressure at surface in hectopascals.
cloudcover_[low/mid/high/total] - Different from the forecast conventions. Cloud cover at 0-3 km, 3-8, 8+, and total.
windspeed_10m - Different from the forecast conventions. The wind speed at 10 meters above ground in meters per second.
winddirection_10m - Different from the forecast conventions. The wind direction at 10 meters above ground in degrees.
shortwave_radiation - Different from the forecast conventions. The global horizontal irradiation in watt-hours per square meter.
direct_solar_radiation
diffuse_radiation - Different from the forecast conventions. The diffuse solar irradiation in watt-hours per square meter.
[latitude/longitude] - The coordinates of the weather station.
data_block_id
"""
historical_weather

,datetime,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,latitude,longitude,data_block_id
0,2021-09-01 00:00:00,14.2,11.6,0.0,0.0,1015.9,31,31,0,11,7.083333,8,0.0,0.0,0.0,57.6,21.7,1.0
1,2021-09-01 00:00:00,13.9,11.5,0.0,0.0,1010.7,33,37,0,0,5.111111,359,0.0,0.0,0.0,57.6,22.2,1.0
2,2021-09-01 00:00:00,14.0,12.5,0.0,0.0,1015.0,31,34,0,0,6.333333,355,0.0,0.0,0.0,57.6,22.7,1.0
3,2021-09-01 00:00:00,14.6,11.5,0.0,0.0,1017.3,0,0,0,0,8.083333,297,358.0,277.0,81.0,57.6,23.2,1.0
4,2021-09-01 00:00:00,15.7,12.9,0.0,0.0,1014.0,22,25,0,0,8.416667,5,0.0,0.0,0.0,57.6,23.7,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1710797,2023-05-30 10:00:00,11.7,4.6,0.0,0.0,1018.9,40,9,54,0,1.055556,253,567.0,392.0,175.0,59.7,26.2,637.0
1710798,2023-05-30 10:00:00,12.3,3.5,0.0,0.0,1019.0,46,4,70,0,0.805556,263,581.0,407.0,174.0,59.7,26.7,637.0
1710799,2023-05-30 10:00:00,9.8,3.0,0.0,0.0,1019.2,41,4,62,0,1.972222,285,609.0,432.0,177.0,59.7,27.2,637.0
1710800,2023-05-30 10:00:00,11.7,1.6,0.0,0.0,1019.0,44,0,73,0,3.500000,307,658.0,521.0,137.0,59.7,27.7,637.0


# Generate Features

In [110]:
class FeatureProcessorClass():
    def __init__(self):         
        # Columns to join on for the different datasets
        self.weather_join = ['datetime', 'county', 'data_block_id']
        self.gas_join = ['data_block_id']
        self.electricity_join = ['datetime', 'data_block_id']
        self.client_join = ['county', 'is_business', 'product_type', 'data_block_id']
        
        # Columns of latitude & longitude
        self.lat_lon_columns = ['latitude', 'longitude']
        
        # Aggregate stats 
        self.agg_stats = ['mean'] #, 'min', 'max', 'std', 'median']
        
        # Categorical columns (specify for XGBoost)
        self.category_columns = ['county', 'is_business', 'product_type', 'is_consumption', 'data_block_id']

    def create_new_column_names(self, df, suffix, columns_no_change):
        '''Change column names by given suffix, keep columns_no_change, and return back the data'''
        df.columns = [col + suffix 
                      if col not in columns_no_change
                      else col
                      for col in df.columns
                      ]
        return df 

    def flatten_multi_index_columns(self, df):
        df.columns = ['_'.join([col for col in multi_col if len(col)>0]) 
                      for multi_col in df.columns]
        return df
    
    def create_data_features(self, data):
        '''📊Create features for main data (test or train) set📊'''
        # To datetime
        data['datetime'] = pd.to_datetime(data['datetime'])
        
        # Time period features
        data['date'] = data['datetime'].dt.normalize()
        data['year'] = data['datetime'].dt.year
        data['quarter'] = data['datetime'].dt.quarter
        data['month'] = data['datetime'].dt.month
        data['week'] = data['datetime'].dt.isocalendar().week
        data['hour'] = data['datetime'].dt.hour
        
        # Day features
        data['day_of_year'] = data['datetime'].dt.day_of_year
        data['day_of_month']  = data['datetime'].dt.day
        data['day_of_week'] = data['datetime'].dt.day_of_week
        return data

    def create_client_features(self, client):
        '''💼 Create client features 💼'''
        # Modify column names - specify suffix
        client = self.create_new_column_names(client, 
                                           suffix='_client',
                                           columns_no_change = self.client_join
                                          )       
        return client
    
    def create_historical_weather_features(self, historical_weather):
        '''⌛🌤️ Create historical weather features 🌤️⌛'''
        
        # To datetime
        historical_weather['datetime'] = pd.to_datetime(historical_weather['datetime'])
        
        # Add county
        historical_weather[self.lat_lon_columns] = historical_weather[self.lat_lon_columns].astype(float).round(1)
        historical_weather = historical_weather.merge(location, how = 'left', on = self.lat_lon_columns)

        # Modify column names - specify suffix
        historical_weather = self.create_new_column_names(historical_weather,
                                                          suffix='_h',
                                                          columns_no_change = self.lat_lon_columns + self.weather_join
                                                          ) 
        
        # Group by & calculate aggregate stats 
        agg_columns = [col for col in historical_weather.columns if col not in self.lat_lon_columns + self.weather_join]
        agg_dict = {agg_col: self.agg_stats for agg_col in agg_columns}
        historical_weather = historical_weather.groupby(self.weather_join).agg(agg_dict).reset_index() 
        
        # Flatten the multi column aggregates
        historical_weather = self.flatten_multi_index_columns(historical_weather) 
        
        # Test set has 1 day offset for hour<11 and 2 day offset for hour>11
        historical_weather['hour_h'] = historical_weather['datetime'].dt.hour
        historical_weather['datetime'] = (historical_weather
                                               .apply(lambda x: 
                                                      x['datetime'] + pd.DateOffset(1) 
                                                      if x['hour_h']< 11 
                                                      else x['datetime'] + pd.DateOffset(2),
                                                      axis=1)
                                              )
        
        return historical_weather
    
    def create_forecast_weather_features(self, forecast_weather):
        '''🔮🌤️ Create forecast weather features 🌤️🔮'''
        
        # Rename column and drop
        forecast_weather = (forecast_weather
                            .rename(columns = {'forecast_datetime': 'datetime'})
                            .drop(columns = 'origin_datetime') # not needed
                           )
        
        # To datetime
        forecast_weather['datetime'] = (pd.to_datetime(forecast_weather['datetime'])
                                        .dt
                                        .tz_localize(None)
                                       )

        # Add county
        forecast_weather[self.lat_lon_columns] = forecast_weather[self.lat_lon_columns].astype(float).round(1)
        forecast_weather = forecast_weather.merge(location, how = 'left', on = self.lat_lon_columns)
        
        # Modify column names - specify suffix
        forecast_weather = self.create_new_column_names(forecast_weather,
                                                        suffix='_f',
                                                        columns_no_change = self.lat_lon_columns + self.weather_join
                                                        ) 
        
        # Group by & calculate aggregate stats 
        agg_columns = [col for col in forecast_weather.columns if col not in self.lat_lon_columns + self.weather_join]
        agg_dict = {agg_col: self.agg_stats for agg_col in agg_columns}
        forecast_weather = forecast_weather.groupby(self.weather_join).agg(agg_dict).reset_index() 
        
        # Flatten the multi column aggregates
        forecast_weather = self.flatten_multi_index_columns(forecast_weather)     
        return forecast_weather

    def create_electricity_features(self, electricity):
        '''⚡ Create electricity prices features ⚡'''
        # To datetime
        electricity['forecast_date'] = pd.to_datetime(electricity['forecast_date'])
        
        # Test set has 1 day offset
        electricity['datetime'] = electricity['forecast_date'] + pd.DateOffset(1)
        
        # Modify column names - specify suffix
        electricity = self.create_new_column_names(electricity, 
                                                   suffix='_electricity',
                                                   columns_no_change = self.electricity_join
                                                  )             
        return electricity

    def create_gas_features(self, gas):
        '''⛽ Create gas prices features ⛽'''
        # Mean gas price
        gas['mean_price_per_mwh'] = (gas['lowest_price_per_mwh'] + gas['highest_price_per_mwh'])/2
        
        # Modify column names - specify suffix
        gas = self.create_new_column_names(gas, 
                                           suffix='_gas',
                                           columns_no_change = self.gas_join
                                          )       
        return gas
    
    def __call__(self, data, client, historical_weather, forecast_weather, electricity, gas):
        '''Processing of features from all datasets, merge together and return features for dataframe df '''
        # Create features for relevant dataset
        data = self.create_data_features(data)
        client = self.create_client_features(client)
        historical_weather = self.create_historical_weather_features(historical_weather)
        forecast_weather = self.create_forecast_weather_features(forecast_weather)
        electricity = self.create_electricity_features(electricity)
        gas = self.create_gas_features(gas)
        
        # 🔗 Merge all datasets into one df 🔗
        df = data.merge(client, how='left', on = self.client_join)
        df = df.merge(historical_weather, how='left', on = self.weather_join)
        df = df.merge(forecast_weather, how='left', on = self.weather_join)
        df = df.merge(electricity, how='left', on = self.electricity_join)
        df = df.merge(gas, how='left', on = self.gas_join)
        
        # Change columns to categorical for XGBoost
        df[self.category_columns] = df[self.category_columns].astype('category')
        return df

In [206]:
%%time

FeatureProcessor = FeatureProcessorClass()

train_with_features = FeatureProcessor(data = train.copy(),
                        client = client.copy(),
                        historical_weather = historical_weather.copy(),
                        forecast_weather = forecast_weather.copy(),
                        electricity = electricity.copy(),
                        gas = gas.copy())

train_with_features

CPU times: user 8.79 s, sys: 2.16 s, total: 11 s
Wall time: 11.2 s


,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,date,...,snowfall_f_mean,total_precipitation_f_mean,forecast_date_electricity,euros_per_mwh_electricity,origin_date_electricity,forecast_date_gas,lowest_price_per_mwh_gas,highest_price_per_mwh_gas,origin_date_gas,mean_price_per_mwh_gas
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0,2021-09-01,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0,2021-09-01,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1,2021-09-01,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1,2021-09-01,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2,2021-09-01,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018347,15,1,0,197.233,1,2023-05-31 23:00:00,637,2018347,64,2023-05-31,...,0.0,0.0,2023-05-30 23:00:00,-1.29,2023-05-29 23:00:00,2023-05-30,29.0,34.0,2023-05-29,31.5
2018348,15,1,1,0.000,0,2023-05-31 23:00:00,637,2018348,59,2023-05-31,...,0.0,0.0,2023-05-30 23:00:00,-1.29,2023-05-29 23:00:00,2023-05-30,29.0,34.0,2023-05-29,31.5
2018349,15,1,1,28.404,1,2023-05-31 23:00:00,637,2018349,59,2023-05-31,...,0.0,0.0,2023-05-30 23:00:00,-1.29,2023-05-29 23:00:00,2023-05-30,29.0,34.0,2023-05-29,31.5
2018350,15,1,3,0.000,0,2023-05-31 23:00:00,637,2018350,60,2023-05-31,...,0.0,0.0,2023-05-30 23:00:00,-1.29,2023-05-29 23:00:00,2023-05-30,29.0,34.0,2023-05-29,31.5


In [207]:
# Remove columns for features
no_features = ['date', 
                'latitude', 
                'longitude', 
                'data_block_id', 
                'row_id',
                'hours_ahead',
                'hour_h',
                'target'
               ]

remove_columns = [col for col in train_with_features.columns for no_feature in no_features if no_feature in col]
features = [col for col in train_with_features.columns if col not in remove_columns]
PrintColor(f'There are {len(features)} features: {features}')

There are 45 features: ['county', 'is_business', 'product_type', 'is_consumption', 'prediction_unit_id', 'year', 'quarter', 'month', 'week', 'hour', 'day_of_year', 'day_of_month', 'day_of_week', 'eic_count_client', 'installed_capacity_client', 'temperature_h_mean', 'dewpoint_h_mean', 'rain_h_mean', 'snowfall_h_mean', 'surface_pressure_h_mean', 'cloudcover_total_h_mean', 'cloudcover_low_h_mean', 'cloudcover_mid_h_mean', 'cloudcover_high_h_mean', 'windspeed_10m_h_mean', 'winddirection_10m_h_mean', 'shortwave_radiation_h_mean', 'direct_solar_radiation_h_mean', 'diffuse_radiation_h_mean', 'temperature_f_mean', 'dewpoint_f_mean', 'cloudcover_high_f_mean', 'cloudcover_low_f_mean', 'cloudcover_mid_f_mean', 'cloudcover_total_f_mean', '10_metre_u_wind_component_f_mean', '10_metre_v_wind_component_f_mean', 'direct_solar_radiation_f_mean', 'surface_solar_radiation_downwards_f_mean', 'snowfall_f_mean', 'total_precipitation_f_mean', 'euros_per_mwh_electricity', 'lowest_price_per_mwh_gas', 'highest_

# Check data quality

In [157]:
def filter_data_with_feature(df, target_country, target_product_type, target_is_business, target_is_consumption, target_data_block_id=None):
    '''Filter data with given target values'''
    df = df[(df['county'] == target_country) &
            (df['product_type'] == target_product_type) &
            (df['is_business'] == target_is_business) &
            (df['is_consumption'] == target_is_consumption)]
    if target_data_block_id:
        df = df[df['data_block_id'] == target_data_block_id]
    return df

def filter_data(df, target_country, target_product_type, target_is_business, target_is_consumption, target_data_block_id=None):
    '''Filter data with given target values'''
    df = df[(df['county'] == target_country) &
            (df['product_type'] == target_product_type) &
            (df['is_business'] == target_is_business) &
            (df['is_consumption'] == target_is_consumption)]
    if target_data_block_id:
        df = df[df['data_block_id'] == target_data_block_id]
    return df

def filter_client(df, target_country, target_product_type, target_is_business, target_data_block_id=None):
    '''Filter data with given target values'''
    df = df[(df['county'] == target_country) &
            (df['product_type'] == target_product_type) &
            (df['is_business'] == target_is_business)]
    if target_data_block_id:
        df = df[df['data_block_id'] == target_data_block_id]
    return df

In [208]:
target_country = 0
target_product_type = 0
target_data_block_id = 2
target_is_business = 1
target_is_consumption = 1

target_train_with_features = filter_data_with_feature(train_with_features, target_country, target_product_type, target_is_business, target_is_consumption, target_data_block_id)
target_train_with_features

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,date,...,snowfall_f_mean,total_precipitation_f_mean,forecast_date_electricity,euros_per_mwh_electricity,origin_date_electricity,forecast_date_gas,lowest_price_per_mwh_gas,highest_price_per_mwh_gas,origin_date_gas,mean_price_per_mwh_gas
5863,0,1,0,64.1,1,2021-09-03 00:00:00,2,5863,3,2021-09-03,...,0.0,0.000094,2021-09-02 00:00:00,96.99,2021-09-01 00:00:00,2021-09-02,45.62,46.29,2021-09-01,45.955
5985,0,1,0,57.0,1,2021-09-03 01:00:00,2,5985,3,2021-09-03,...,0.0,0.000094,2021-09-02 01:00:00,94.77,2021-09-01 01:00:00,2021-09-02,45.62,46.29,2021-09-01,45.955
6107,0,1,0,56.8,1,2021-09-03 02:00:00,2,6107,3,2021-09-03,...,0.0,0.000068,2021-09-02 02:00:00,93.88,2021-09-01 02:00:00,2021-09-02,45.62,46.29,2021-09-01,45.955
6229,0,1,0,59.2,1,2021-09-03 03:00:00,2,6229,3,2021-09-03,...,0.0,0.000054,2021-09-02 03:00:00,92.25,2021-09-01 03:00:00,2021-09-02,45.62,46.29,2021-09-01,45.955
6351,0,1,0,63.3,1,2021-09-03 04:00:00,2,6351,3,2021-09-03,...,0.0,0.000042,2021-09-02 04:00:00,91.83,2021-09-01 04:00:00,2021-09-02,45.62,46.29,2021-09-01,45.955
6473,0,1,0,76.8,1,2021-09-03 05:00:00,2,6473,3,2021-09-03,...,0.0,0.000050,2021-09-02 05:00:00,98.62,2021-09-01 05:00:00,2021-09-02,45.62,46.29,2021-09-01,45.955
6595,0,1,0,74.1,1,2021-09-03 06:00:00,2,6595,3,2021-09-03,...,0.0,0.000032,2021-09-02 06:00:00,128.94,2021-09-01 06:00:00,2021-09-02,45.62,46.29,2021-09-01,45.955
6717,0,1,0,103.2,1,2021-09-03 07:00:00,2,6717,3,2021-09-03,...,0.0,0.000086,2021-09-02 07:00:00,150.00,2021-09-01 07:00:00,2021-09-02,45.62,46.29,2021-09-01,45.955
6839,0,1,0,124.9,1,2021-09-03 08:00:00,2,6839,3,2021-09-03,...,0.0,0.000177,2021-09-02 08:00:00,146.38,2021-09-01 08:00:00,2021-09-02,45.62,46.29,2021-09-01,45.955
6961,0,1,0,124.4,1,2021-09-03 09:00:00,2,6961,3,2021-09-03,...,0.0,0.000218,2021-09-02 09:00:00,130.80,2021-09-01 09:00:00,2021-09-02,45.62,46.29,2021-09-01,45.955


In [209]:
target_train_with_features[col_train]

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
5863,0,1,0,64.1,1,2021-09-03 00:00:00,2,5863,3
5985,0,1,0,57.0,1,2021-09-03 01:00:00,2,5985,3
6107,0,1,0,56.8,1,2021-09-03 02:00:00,2,6107,3
6229,0,1,0,59.2,1,2021-09-03 03:00:00,2,6229,3
6351,0,1,0,63.3,1,2021-09-03 04:00:00,2,6351,3
6473,0,1,0,76.8,1,2021-09-03 05:00:00,2,6473,3
6595,0,1,0,74.1,1,2021-09-03 06:00:00,2,6595,3
6717,0,1,0,103.2,1,2021-09-03 07:00:00,2,6717,3
6839,0,1,0,124.9,1,2021-09-03 08:00:00,2,6839,3
6961,0,1,0,124.4,1,2021-09-03 09:00:00,2,6961,3


In [210]:
target_train_with_features[col_client]

KeyError: "['eic_count', 'installed_capacity'] not in index"

In [201]:
"""
        self.weather_join = ['datetime', 'county', 'data_block_id']
        self.gas_join = ['data_block_id']
        self.electricity_join = ['datetime', 'data_block_id']
"""

def filter_historical_weather(df, target_country, target_data_block_id=None):
    '''Filter data with given target values'''
    df = df[(df['county'] == target_country)]
    if target_data_block_id:
        df = df[df['data_block_id'] == target_data_block_id]
    return df

#target_historical_weather = filter_historical_weather(historical_weather, target_country, target_data_block_id)
historical_weather

,datetime,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,latitude,longitude,data_block_id
0,2021-09-01 00:00:00,14.2,11.6,0.0,0.0,1015.9,31,31,0,11,7.083333,8,0.0,0.0,0.0,57.6,21.7,1.0
1,2021-09-01 00:00:00,13.9,11.5,0.0,0.0,1010.7,33,37,0,0,5.111111,359,0.0,0.0,0.0,57.6,22.2,1.0
2,2021-09-01 00:00:00,14.0,12.5,0.0,0.0,1015.0,31,34,0,0,6.333333,355,0.0,0.0,0.0,57.6,22.7,1.0
3,2021-09-01 00:00:00,14.6,11.5,0.0,0.0,1017.3,0,0,0,0,8.083333,297,358.0,277.0,81.0,57.6,23.2,1.0
4,2021-09-01 00:00:00,15.7,12.9,0.0,0.0,1014.0,22,25,0,0,8.416667,5,0.0,0.0,0.0,57.6,23.7,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1710797,2023-05-30 10:00:00,11.7,4.6,0.0,0.0,1018.9,40,9,54,0,1.055556,253,567.0,392.0,175.0,59.7,26.2,637.0
1710798,2023-05-30 10:00:00,12.3,3.5,0.0,0.0,1019.0,46,4,70,0,0.805556,263,581.0,407.0,174.0,59.7,26.7,637.0
1710799,2023-05-30 10:00:00,9.8,3.0,0.0,0.0,1019.2,41,4,62,0,1.972222,285,609.0,432.0,177.0,59.7,27.2,637.0
1710800,2023-05-30 10:00:00,11.7,1.6,0.0,0.0,1019.0,44,0,73,0,3.500000,307,658.0,521.0,137.0,59.7,27.7,637.0


In [135]:
'''
An unlocked version of the timeseries API intended for testing alternate inputs.
Mirrors the production timeseries API in the crucial respects, but won't be as fast.

ONLY works afer the first three variables in MockAPI.__init__ are populated.
'''

from typing import Sequence, Tuple

import pandas as pd


class MockApi:
    def __init__(self):
        '''
        YOU MUST UPDATE THE FIRST THREE LINES of this method.
        They've been intentionally left in an invalid state.

        Variables to set:
            input_paths: a list of two or more paths to the csv files to be served
            group_id_column: the column that identifies which groups of rows the API should serve.
                A call to iter_test serves all rows of all dataframes with the current group ID value.
            export_group_id_column: if true, the dataframes iter_test serves will include the group_id_column values.
        '''
             
        self.fpath=f"{BASE_INPUT_PATH}/predict-energy-behavior-of-prosumers/example_test_files"
        self.input_paths: Sequence[str] = [self.fpath + '/test.csv', self.fpath + '/revealed_targets.csv', 
                                           self.fpath + '/client.csv', 
                                           self.fpath + '/historical_weather.csv', self.fpath + '/forecast_weather.csv', 
                                           self.fpath + '/electricity_prices.csv', self.fpath + '/gas_prices.csv',
                                           self.fpath + '/sample_submission.csv']
        self.group_id_column: str = "data_block_id"
        self.export_group_id_column: bool = True
        # iter_test is only designed to support at least two dataframes, such as test and sample_submission
        assert len(self.input_paths) >= 2

        self._status = 'initialized'
        self.predictions = []

    def iter_test(self) -> Tuple[pd.DataFrame]:
        '''
        Loads all of the dataframes specified in self.input_paths,
        then yields all rows in those dataframes that equal the current self.group_id_column value.
        '''
        if self._status != 'initialized':

            raise Exception('WARNING: the real API can only iterate over `iter_test()` once.')

        dataframes = []
        for pth in self.input_paths:
            dataframes.append(pd.read_csv(pth, low_memory=False))
        group_order = dataframes[0][self.group_id_column].drop_duplicates().tolist()
        dataframes = [df.set_index(self.group_id_column) for df in dataframes]

        for group_id in group_order:
            self._status = 'prediction_needed'
            current_data = []
            for df in dataframes:
                cur_df = df.loc[group_id].copy()
                # returning single line dataframes from df.loc requires special handling
                if not isinstance(cur_df, pd.DataFrame):
                    cur_df = pd.DataFrame({a: b for a, b in zip(cur_df.index.values, cur_df.values)}, index=[group_id])
                    cur_df.index.name = self.group_id_column
                cur_df = cur_df.reset_index(drop=not(self.export_group_id_column))
                current_data.append(cur_df)
            yield tuple(current_data)

            while self._status != 'prediction_received':
                print('You must call `predict()` successfully before you can continue with `iter_test()`', flush=True)
                yield None

        with open('submission.csv', 'w') as f_open:
            pd.concat(self.predictions).to_csv(f_open, index=False)
        self._status = 'finished'

    def predict(self, user_predictions: pd.DataFrame):
        '''
        Accepts and stores the user's predictions and unlocks iter_test once that is done
        '''
        if self._status == 'finished':
            raise Exception('You have already made predictions for the full test set.')
        if self._status != 'prediction_needed':
            raise Exception('You must get the next test sample from `iter_test()` first.')
        if not isinstance(user_predictions, pd.DataFrame):
            raise Exception('You must provide a DataFrame.')

        self.predictions.append(user_predictions)
        self._status = 'prediction_received'


def make_env():
    return MockApi()

In [213]:
env = make_env()
iter_test = env.iter_test()

target_country = 0
target_product_type = 0
#target_data_block_id = 0
target_is_business = 1
target_is_consumption = 1

cache_test = pd.DataFrame()
cache_revealed_targets = pd.DataFrame()

for (
    test, 
    revealed_targets, 
    client, 
    historical_weather,
    forecast_weather, 
    electricity_prices, 
    gas_prices, 
    sample_prediction
) in iter_test:
    
    cache_test = pd.concat([cache_test, test])
    cache_revealed_targets = pd.concat([cache_revealed_targets, revealed_targets])

    cond_test = (cache_test['county'] == target_country) & (cache_test["is_business"] == target_is_business) & (cache_test['product_type'] == target_product_type) & (cache_test['is_consumption'] == target_is_consumption)
    display_df(cache_test[cond_test], "test", 100)

    # lag 2 days
    cond_revealed_targets = (cache_revealed_targets['county'] == target_country) & (cache_revealed_targets["is_business"] == target_is_business) & (cache_revealed_targets['product_type'] == target_product_type) & (cache_revealed_targets['is_consumption'] == target_is_consumption)
    display_df(cache_revealed_targets[cond_revealed_targets], "revealed_targets", 100)

    break

test data has 24 rows and 9 columns. 
 ===>  row:


,data_block_id,county,is_business,product_type,is_consumption,prediction_datetime,row_id,prediction_unit_id,currently_scored
7,634,0,1,0,1,2023-05-28 00:00:00,2005879,3,False
137,634,0,1,0,1,2023-05-28 01:00:00,2006009,3,False
267,634,0,1,0,1,2023-05-28 02:00:00,2006139,3,False
397,634,0,1,0,1,2023-05-28 03:00:00,2006269,3,False
527,634,0,1,0,1,2023-05-28 04:00:00,2006399,3,False
657,634,0,1,0,1,2023-05-28 05:00:00,2006529,3,False
787,634,0,1,0,1,2023-05-28 06:00:00,2006659,3,False
917,634,0,1,0,1,2023-05-28 07:00:00,2006789,3,False
1047,634,0,1,0,1,2023-05-28 08:00:00,2006919,3,False
1177,634,0,1,0,1,2023-05-28 09:00:00,2007049,3,False


revealed_targets data has 24 rows and 9 columns. 
 ===>  row:


,data_block_id,county,is_business,product_type,target,is_consumption,datetime,row_id,prediction_unit_id
7,634,0,1,0,468.699,1,2023-05-26 00:00:00,1999543,3
139,634,0,1,0,464.341,1,2023-05-26 01:00:00,1999675,3
271,634,0,1,0,446.274,1,2023-05-26 02:00:00,1999807,3
403,634,0,1,0,460.645,1,2023-05-26 03:00:00,1999939,3
535,634,0,1,0,427.679,1,2023-05-26 04:00:00,2000071,3
667,634,0,1,0,409.314,1,2023-05-26 05:00:00,2000203,3
799,634,0,1,0,500.263,1,2023-05-26 06:00:00,2000335,3
931,634,0,1,0,744.836,1,2023-05-26 07:00:00,2000467,3
1063,634,0,1,0,836.245,1,2023-05-26 08:00:00,2000599,3
1195,634,0,1,0,1026.682,1,2023-05-26 09:00:00,2000731,3
